## Imbalanced classification-사고 주변환경정보를 활용하여 사망사고 예측
1. 머신러닝 목표 :
2. 모델 Features and Target
3. 적용할 모델

In [15]:
#데이터 핸들링 라이브러리
import pandas as pd
import numpy as np
#데이터 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.family':'AppleGothic'})
mpl.rc('axes', unicode_minus=False)
%config InlineBackend.figure_format = 'retina'


## 용어 정리
- Imbalance 정도
    - Slight Imbalance: An imbalanced classification problem where the distribution of examples is uneven by a small
    amount in the training dataset (e.g. 4:6)
    - Severe Imbalance: An imbalanced classification problem where the distribution of examples is uneven by a large
    amount in the training dataset (e.g. 1:100 or more)
- class의 분류
    - Majority Class: The class (or classes) in an imbalanced classification predictive modeling problem that has
    many examples. (여기선 사망하지 않음(0))
    - Minority Class: The class in an imbalanced classification predictive modeling problem that has few examples.
    (여기선 사망함(1))

- Imbalance Classification 예
    - Fraud Detection.
    - Claim Prediction
    - Default Prediction.
    - Churn Prediction.
    - Spam Detection.
    - Anomaly Detection.
    - Outlier Detection.
    - Intrusion Detection
    - Conversion Prediction.

### Imbalance Classification에서 정확도accuracy가 좋은 척도일까?
- 아님!
- 만약 1000명중 한 명이 걸리는 회귀병이 있는데 잠복기동안 아무 증상이 없다 갑자기 죽는 병이라 하자.
    - 이 병을 밝힐 수 있는 장치는 진단기 A밖에 없다. 고 하자 이 진단기 A가 어느날 고장이 나서 무조건 음성판정만을 내렸다고 하자.
    - 그리고 이 고장난 사실을 아무도 모른채 계속 사용되었다고 하자. 이 기계는 999/1000의 높은 정확도를 가진 기계였지만, 회귀병을 탐지한다는 기계 본래의 목적에는 전혀 부합하지 못한 고철덩어리에 불과했던 것이다 (비유 출처: 본인)
    - 양성인 사실을 놓치는 것이 심각한 생명,신체상의 손해를 끼칠수 있는 경우 정확도accuracy가 높은것 보다도 재현율recall(실제 양성인 경우에서 양성이라 판정되는 비율)이 더 중요하다 볼수 있다
    - [참고](https://en.wikipedia.org/wiki/Accuracy_paradox)
### 레퍼런스
- [imbalanced classification](https://machinelearningmastery.com/what-is-imbalanced-classification/)

## 데이터 전처리 계획
1. 데이터 불러오기, 결측치 처리하기
2. 상대적으로 중요성이 떨어져보이는 피처 삭제, 필요하면 피처명을 직관적으로 다시 네이밍하기
3. 비숫자-문자열 독립변수 인코딩하기
4. 데이터 정규화
5. 훈련-테스트셋 분리

## 모델 학습 계획
1. 모델 생성
2. Cross_validation을 통한 모델학습
3. Accuracy, Precision,Recall, ROC 4개의 지표로 분류모델의 정확도 평가하기

In [16]:
from preprocessing_data_for_unsupervised import f1_read
df= f1_read()
df

데이터 불러오기 및 통합에 걸린 시간은 62.96861290931702 입니다


,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2020010100100011,2020년 1월 1일 00시,수요일,대구광역시 달서구 죽전동,중상사고,0,1,0,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,70세,상해없음,이륜,남,18세,중상
1,2020010100100048,2020년 1월 1일 02시,수요일,대구광역시 달서구 송현동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,원동기,남,50세,상해없음,승용,남,26세,경상
2,2020010100100071,2020년 1월 1일 05시,수요일,대구광역시 북구 대현동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,교차로 - 교차로부근,기타불명,기타불명,미분류,기타불명,승용,남,37세,경상
3,2020010100100072,2020년 1월 1일 05시,수요일,대구광역시 중구 삼덕동1가,경상사고,0,0,1,0,차대사람 - 길가장자리구역통행중,...,맑음,단일로 - 기타,승용,남,18세,상해없음,보행자,남,21세,경상
4,2020010100100087,2020년 1월 1일 06시,수요일,대구광역시 수성구 두산동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,단일로 - 기타,승용,남,45세,상해없음,승용,남,26세,경상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412779,2021123100100475,2021년 12월 31일 18시,금요일,울산광역시 북구 호계동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,교차로 - 교차로횡단보도내,승용,남,33세,상해없음,보행자,남,65세,중상
412780,2021123100100476,2021년 12월 31일 18시,금요일,울산광역시 북구 중산동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,단일로 - 기타,승용,남,38세,상해없음,승용,남,32세,경상
412781,2021123100100477,2021년 12월 31일 18시,금요일,울산광역시 북구 진장동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,71세,상해없음,승용,여,30세,경상
412782,2021123100100593,2021년 12월 31일 21시,금요일,울산광역시 남구 야음동,경상사고,0,0,1,0,차대차 - 후진중충돌,...,맑음,단일로 - 기타,승용,남,28세,상해없음,이륜,남,17세,경상


In [17]:
"""
결측치 확인하기 => 피해운전자의 차종,성별,연령,상해정도를 파악하지 못하는 데이터가 16704건이 존재함
원본데이터 412784에 비하면 삭제해도 학습에 영향력을 크게 미치지 못할 뿐더러, 이러한 데이터들이
특정 편향성을 갖고 존재할 것이라 보기 어렵다(아마도 기록 누락에 의해 이러한 데이터가 존재할 것이기 때문)
그렇기 때문에 데이터를 제거해도 될것이라 판단하였음
"""
from preprocessing_data_for_unsupervised import f2_removeNaN
df= f2_removeNaN(df)
df

사고번호              0
사고일시              0
요일                0
시군구               0
사고내용              0
사망자수              0
중상자수              0
경상자수              0
부상신고자수            0
사고유형              0
법규위반              0
노면상태              0
기상상태              0
도로형태              0
가해운전자 차종          0
가해운전자 성별          0
가해운전자 연령          0
가해운전자 상해정도        0
피해운전자 차종      16704
피해운전자 성별      16706
피해운전자 연령      16704
피해운전자 상해정도    16706
dtype: int64


,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2020010100100011,2020년 1월 1일 00시,수요일,대구광역시 달서구 죽전동,중상사고,0,1,0,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,70세,상해없음,이륜,남,18세,중상
1,2020010100100048,2020년 1월 1일 02시,수요일,대구광역시 달서구 송현동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,원동기,남,50세,상해없음,승용,남,26세,경상
2,2020010100100071,2020년 1월 1일 05시,수요일,대구광역시 북구 대현동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,교차로 - 교차로부근,기타불명,기타불명,미분류,기타불명,승용,남,37세,경상
3,2020010100100072,2020년 1월 1일 05시,수요일,대구광역시 중구 삼덕동1가,경상사고,0,0,1,0,차대사람 - 길가장자리구역통행중,...,맑음,단일로 - 기타,승용,남,18세,상해없음,보행자,남,21세,경상
4,2020010100100087,2020년 1월 1일 06시,수요일,대구광역시 수성구 두산동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,단일로 - 기타,승용,남,45세,상해없음,승용,남,26세,경상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412779,2021123100100475,2021년 12월 31일 18시,금요일,울산광역시 북구 호계동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,교차로 - 교차로횡단보도내,승용,남,33세,상해없음,보행자,남,65세,중상
412780,2021123100100476,2021년 12월 31일 18시,금요일,울산광역시 북구 중산동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,단일로 - 기타,승용,남,38세,상해없음,승용,남,32세,경상
412781,2021123100100477,2021년 12월 31일 18시,금요일,울산광역시 북구 진장동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,71세,상해없음,승용,여,30세,경상
412782,2021123100100593,2021년 12월 31일 21시,금요일,울산광역시 남구 야음동,경상사고,0,0,1,0,차대차 - 후진중충돌,...,맑음,단일로 - 기타,승용,남,28세,상해없음,이륜,남,17세,경상


In [18]:
#데이터 데이터타입, 데이터갯수 확인하기
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396078 entries, 0 to 412783
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   사고번호        396078 non-null  int64 
 1   사고일시        396078 non-null  object
 2   요일          396078 non-null  object
 3   시군구         396078 non-null  object
 4   사고내용        396078 non-null  object
 5   사망자수        396078 non-null  int64 
 6   중상자수        396078 non-null  int64 
 7   경상자수        396078 non-null  int64 
 8   부상신고자수      396078 non-null  int64 
 9   사고유형        396078 non-null  object
 10  법규위반        396078 non-null  object
 11  노면상태        396078 non-null  object
 12  기상상태        396078 non-null  object
 13  도로형태        396078 non-null  object
 14  가해운전자 차종    396078 non-null  object
 15  가해운전자 성별    396078 non-null  object
 16  가해운전자 연령    396078 non-null  object
 17  가해운전자 상해정도  396078 non-null  object
 18  피해운전자 차종    396078 non-null  object
 19  피해운전자 성별    396078 non-

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2020010100100011,2020년 1월 1일 00시,수요일,대구광역시 달서구 죽전동,중상사고,0,1,0,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,70세,상해없음,이륜,남,18세,중상
1,2020010100100048,2020년 1월 1일 02시,수요일,대구광역시 달서구 송현동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,원동기,남,50세,상해없음,승용,남,26세,경상
2,2020010100100071,2020년 1월 1일 05시,수요일,대구광역시 북구 대현동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,교차로 - 교차로부근,기타불명,기타불명,미분류,기타불명,승용,남,37세,경상
3,2020010100100072,2020년 1월 1일 05시,수요일,대구광역시 중구 삼덕동1가,경상사고,0,0,1,0,차대사람 - 길가장자리구역통행중,...,맑음,단일로 - 기타,승용,남,18세,상해없음,보행자,남,21세,경상
4,2020010100100087,2020년 1월 1일 06시,수요일,대구광역시 수성구 두산동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,단일로 - 기타,승용,남,45세,상해없음,승용,남,26세,경상


## 전체사고 중 사망사고 비율 확인하기

In [19]:
from preprocessing_data_for_unsupervised import f3_ratio
f3_ratio(df)

경상사고      276736
중상사고       97602
부상신고사고     17285
사망사고        4455
Name: 사고내용, dtype: int64
전체 사고중 사망사고의 비율은 1.124778452729008% 입니다


## 컬럼 전처리하기
- 불필요해보이는 컬럼 제거하기
    1. 사고번호는 각 사고에 대해 일대일 대응되는 key값으로 나중에 데이터를 결합시키는데 사용될 수 있어 보이나, 일단
    지금 당장 사용되지 않으므로 삭제한다
    2. 사망자수,중상자수,경상자수,부상신고자는 '사고내용'에 필요한 정보가 담겨있으므로 삭제한다
    3.차대차인지 차대사람인지에 대한 정보가 사고유형에 담기므로 가해운전자 차종과 피해운전자 차종에서 삭제하였다
    3. 위치정보는 사용하지 않으므로 시군구도 삭제한다
- 이진분류로 단순화하기
    - 종속변수 '사고내용'에서 사망자를 1, 중상,경상을 0으로 하는 새로운 종속변수 열 '사망사고여부' 을 만들기
    - 발생년월시에서 시간만 떼어내고, 정수형으로 데이터 변환하고, 22~06시를 야간, 그외 시간을 주간으로 분류하는 독립변수 열 '야간여부' 만들기
    - '야간여부'와 '사망사고여부'를 정수로 데이터변환하기
    - 요일은 주말과 평일로 통합하기
- 피해운전자, 가해운전자 연령에서 '세' 단위 떼어내고 정수형으로 데이터 변환하기

In [20]:
from preprocessing_data_for_unsupervised import f4_preprocess
df= f4_preprocess(df)
df

,주말여부,야간여부,사고유형,법규위반,기상상태,도로형태,가해운전자 성별- 여성0 남성1,가해운전자 연령(정규화 됨),피해운전자 성별- 여성0 남성1,피해운전자 연령(정규화 됨),사망사고여부
0,0,0,차대차 - 측면충돌,안전운전불이행,맑음,단일로 - 기타,1,1.384428,1,-1.676547,0
1,0,0,차대차 - 측면충돌,교차로운행방법위반,맑음,교차로 - 교차로안,1,0.117883,1,-1.192179,0
3,0,0,차대사람 - 길가장자리구역통행중,안전운전불이행,맑음,단일로 - 기타,1,-1.908588,1,-1.494909,0
4,0,0,차대차 - 기타,안전운전불이행,맑음,단일로 - 기타,1,-0.198753,1,-1.192179,0
5,0,0,차대차 - 기타,불법유턴,맑음,단일로 - 기타,1,1.511082,1,0.442566,0
...,...,...,...,...,...,...,...,...,...,...,...
412779,0,0,차대사람 - 횡단중,보행자보호의무위반,맑음,교차로 - 교차로횡단보도내,1,-0.958679,1,1.169119,0
412780,0,0,차대차 - 추돌,안전운전불이행,맑음,단일로 - 기타,1,-0.642043,1,-0.828902,0
412781,0,0,차대차 - 측면충돌,차로위반,맑음,단일로 - 기타,1,1.447755,0,-0.949994,0
412782,0,0,차대차 - 후진중충돌,안전운전불이행,맑음,단일로 - 기타,1,-1.275315,1,-1.737093,0


## 범주형 변수 인코딩하기
- 랜덤 포레스트나 트리 기반 알고리즘의 경우 LabelEncdoer를 활용한 인코딩을 하여도 되나, 그 외의 경우 One-Hot Encoding을 하여야 한다
- One-Hot Encoding으로 sklearn에도 모듈이 있지만 pd.get_dummies가 사용편의성이 좋은듯
- [아 졸려](https://blog.roboflow.com/precision-and-recall/#precision-and-recall-examples)

In [21]:
from preprocessing_data_for_unsupervised import f5_encdoding_OneHot
df= f5_encdoding_OneHot(df)
df

,주말여부,야간여부,가해운전자 성별- 여성0 남성1,가해운전자 연령(정규화 됨),피해운전자 성별- 여성0 남성1,피해운전자 연령(정규화 됨),사망사고여부,사고유형_차대사람 - 기타,사고유형_차대사람 - 길가장자리구역통행중,사고유형_차대사람 - 보도통행중,...,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 철길건널목,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장
0,0,0,1,1.384428,1,-1.676547,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,1,0.117883,1,-1.192179,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,-1.908588,1,-1.494909,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,1,-0.198753,1,-1.192179,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,1,1.511082,1,0.442566,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412779,0,0,1,-0.958679,1,1.169119,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
412780,0,0,1,-0.642043,1,-0.828902,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
412781,0,0,1,1.447755,0,-0.949994,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
412782,0,0,1,-1.275315,1,-1.737093,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


## 이상치 데이터 제거하기
- 데이터를 작은값부터 큰값으로 나열한다고 하자 이러한 데이터의 크기순서를 0%~100%로 표기한다
- 25~75%구간을 IQR(Inter Quantile Range)이라 정의한다
- 이상치는 임의로 정의하는데 보통 사용하는 방법은 최솟값에서 1.5IQR를 뺀값을, 최댓값에서 1.5IQR을 더한값을 경계로 하여 그 경계 바깥에 있는 데이터값을 이상치로 간주하는 것이다

In [22]:
from preprocessing_data_for_unsupervised import f6_drop_outlier
df= f6_drop_outlier(df)
df

,주말여부,야간여부,가해운전자 성별- 여성0 남성1,가해운전자 연령(정규화 됨),피해운전자 성별- 여성0 남성1,피해운전자 연령(정규화 됨),사망사고여부,사고유형_차대사람 - 기타,사고유형_차대사람 - 길가장자리구역통행중,사고유형_차대사람 - 보도통행중,...,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 철길건널목,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장
0,0,0,1,1.384428,1,-1.676547,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,1,0.117883,1,-1.192179,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,-1.908588,1,-1.494909,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,1,-0.198753,1,-1.192179,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,1,1.511082,1,0.442566,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412779,0,0,1,-0.958679,1,1.169119,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
412780,0,0,1,-0.642043,1,-0.828902,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
412781,0,0,1,1.447755,0,-0.949994,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
412782,0,0,1,-1.275315,1,-1.737093,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [23]:
df[df['사망사고여부']==0]

,주말여부,야간여부,가해운전자 성별- 여성0 남성1,가해운전자 연령(정규화 됨),피해운전자 성별- 여성0 남성1,피해운전자 연령(정규화 됨),사망사고여부,사고유형_차대사람 - 기타,사고유형_차대사람 - 길가장자리구역통행중,사고유형_차대사람 - 보도통행중,...,도로형태_교차로 - 교차로횡단보도내,도로형태_기타 - 기타,도로형태_단일로 - 고가도로위,도로형태_단일로 - 교량,도로형태_단일로 - 기타,도로형태_단일로 - 지하차도(도로)내,도로형태_단일로 - 철길건널목,도로형태_단일로 - 터널,도로형태_미분류 - 미분류,도로형태_주차장 - 주차장
0,0,0,1,1.384428,1,-1.676547,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,1,0.117883,1,-1.192179,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,-1.908588,1,-1.494909,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,1,-0.198753,1,-1.192179,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,1,1.511082,1,0.442566,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412779,0,0,1,-0.958679,1,1.169119,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
412780,0,0,1,-0.642043,1,-0.828902,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
412781,0,0,1,1.447755,0,-0.949994,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
412782,0,0,1,-1.275315,1,-1.737093,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


 # 데이터 쪼개기

In [24]:
from preprocessing_data_for_unsupervised import f7_divide_dataset_for_oneSVM
X_neg_train,X_test,y_test= f7_divide_dataset_for_oneSVM(df)

X_neg_train의 크기는 307714, y의 크기는 307714
X_test의 크기는 77764, y의 크기는 77764


In [25]:
y_test.sum()

835

## 머신러닝 모듈 정리해두기

In [26]:
""""Binarizer: 문턱값(threshold)를 설정하여 그 이상값을 입력받으면 1을, 아니면 0을 반환하는 함수.
보통 pred_proba란 함수가 특정 카테고리에 대한 확률값을 제공하는 함수인데, 지금 이 예제와 같이 y의 0과 1의 비율이 크게 차이가
나는 경우 1에 대한 예측 확률이 크게 작다. 원래는 1에 대한 pred_proba 예측이 0.5이상 일때 1을 예측하는데, Binarizer를 통해
1을 예측하기 위한 확률 문턱값을 낮출수 있고, 그를 통해 재현율(실제 1일일때 1을 예측하는 비율)을 높일 수 있다"""
from sklearn.preprocessing import Binarizer

#GridSearchCV 라이브러리(HyperParameter를 일일히 대입하여 최적의 파라미터를 찾는 알고리즘)
from sklearn.model_selection import GridSearchCV

#머신러닝 성능 지표 관련 라이브러리
from sklearn.metrics import accuracy_score,precision_score,roc_auc_score,recall_score

#피처 중요도를 순위메기기 위해 사용하는 모듈
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer

#SVM 라이브러리
from sklearn import svm
from sklearn.svm import OneClassSVM
from sklearn.linear_model import SGDOneClassSVM
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline
from sklearn import linear_model

#Isolation Forest
from sklearn.ensemble import IsolationForest

## SGD One-class SVM

In [27]:
# One-class- SVM hyperparameters
nu = 0.0125
gamma = 'auto'

# Fit the SGD One-Class SVM
OCSVM_clf = linear_model.SGDOneClassSVM(nu=nu,verbose=True)
OCSVM_clf.fit(X_neg_train)
y_pred = OCSVM_clf.predict(X_test)


#예측과 실제 데이터값 비교
T_F_lst=[]
for i in range(len(y_pred)):
    T_F_lst.append(y_test.values[i]== y_pred[i])

#precision값 측정하기
print(f'precision은 {np.array(T_F_lst).sum()/np.array(y_pred).sum()} 입니다')

#recall 값 측정하기
N_positive=(y_test.values).sum()
print(f'recall은 {np.array(T_F_lst).sum()/N_positive} 입니다')


-- Epoch 1
Norm: 1.60, NNZs: 46, Bias: -0.420456, T: 307714, Avg. loss: 0.001380
Total training time: 0.05 seconds.
-- Epoch 2
Norm: 1.61, NNZs: 46, Bias: -0.429745, T: 615428, Avg. loss: 0.001402
Total training time: 0.10 seconds.
-- Epoch 3
Norm: 1.62, NNZs: 46, Bias: -0.433790, T: 923142, Avg. loss: 0.001391
Total training time: 0.16 seconds.
-- Epoch 4
Norm: 1.62, NNZs: 46, Bias: -0.436607, T: 1230856, Avg. loss: 0.001420
Total training time: 0.21 seconds.
-- Epoch 5
Norm: 1.62, NNZs: 46, Bias: -0.438992, T: 1538570, Avg. loss: 0.001412
Total training time: 0.25 seconds.
-- Epoch 6
Norm: 1.62, NNZs: 46, Bias: -0.439952, T: 1846284, Avg. loss: 0.001406
Total training time: 0.30 seconds.
Convergence after 6 epochs took 0.30 seconds
precision은 0.010824049932843485 입니다
recall은 0.9844311377245509 입니다


## Isolation Forest

In [28]:
Isolation_Forest = IsolationForest(verbose=True)
Isolation_Forest.fit(X_neg_train)
y_pred = Isolation_Forest.predict(X_test)


#예측과 실제 데이터값 비교
T_F_lst=[]
for i in range(len(y_pred)):
    T_F_lst.append(y_test.values[i]== y_pred[i])

#precision값 측정하기
print(f'precision은 {np.array(T_F_lst).sum()/np.array(y_pred).sum()} 입니다')

#recall 값 측정하기
N_positive=(y_test.values).sum()
print(f'recall은 {np.array(T_F_lst).sum()/N_positive} 입니다')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s finished


precision은 0.010970317188169739 입니다
recall은 0.9808383233532935 입니다
